# LEZIONE 13 - Pipeline di Scikit-Learn

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. **Comprendere** perche le Pipeline sono essenziali nei progetti reali
2. **Costruire** Pipeline che combinano preprocessing e modello
3. **Usare** ColumnTransformer per gestire feature diverse
4. **Evitare** il data leakage con preprocessing corretto
5. **Strutturare** workflow di ML riproducibili e manutenibili

## Prerequisiti

| Concetto | Dove lo trovi |
|----------|---------------|
| Train/Test Split | Lezione 9 |
| Cross-Validation | Lezione 10 |
| Overfitting e Data Leakage | Lezione 12 |
| StandardScaler e preprocessing | Lezione 9 |

## Indice

1. SEZIONE 1 - Teoria
2. SEZIONE 2 - Mappa Mentale
3. SEZIONE 3 - Quaderno Dimostrativo
4. SEZIONE 4 - Metodi Principali
5. SEZIONE 5 - Glossario
6. SEZIONE 6 - Errori Comuni
7. SEZIONE 7 - Conclusione
8. SEZIONE 8 - Checklist
9. SEZIONE 9 - Changelog

---

## Librerie Utilizzate

```python
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
```

# SEZIONE 1 - Teoria

## 1.1 Il Problema: Perche Servono le Pipeline?

### Scenario Tipico (SBAGLIATO)

```python
# ERRORE COMUNE: preprocessing su TUTTI i dati prima dello split

# 1. Carichiamo i dati
X = load_data()

# 2. Preprocessing su TUTTO il dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # ERRORE! fit su tutti i dati

# 3. Poi facciamo lo split
X_train, X_test = train_test_split(X_scaled)

# 4. Alleniamo il modello
model.fit(X_train, y_train)
```

### Perche e Sbagliato? DATA LEAKAGE

```
            PRIMA DELLO SPLIT
            -----------------
               +----------+
    Tutti i -> |  Scaler  | -> fit() vede ANCHE i dati di test!
    dati       |   fit()  |
               +----------+
                    |
            +-------+-------+
            v               v
         Train            Test
         
Il modello "bara": ha visto informazioni sul test set!
```

**Conseguenze:**
- Le performance sul test set sono **ottimistiche**
- In produzione il modello avra performance **peggiori**
- Hai violato la regola fondamentale del ML

### Scenario Corretto (MA SCOMODO)

```python
# CORRETTO ma verboso e error-prone

# 1. Prima lo split
X_train, X_test = train_test_split(X)

# 2. Fit SOLO su train
scaler = StandardScaler()
scaler.fit(X_train)

# 3. Transform su entrambi
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Stessa cosa per ogni step di preprocessing...
imputer = SimpleImputer()
imputer.fit(X_train_scaled)
X_train_imputed = imputer.transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

# 5. Allena modello
model.fit(X_train_imputed, y_train)

# 6. Per predizioni su nuovi dati: ripeti tutti i transform!
```

---

## 1.2 La Soluzione: sklearn.pipeline.Pipeline

Una **Pipeline** e una sequenza ordinata di trasformazioni + un modello finale, incapsulati in un unico oggetto.

```
               PIPELINE
    +----------------------------------+
    |                                  |
    |  Step 1      Step 2      Step 3  |
    | +--------+  +--------+  +-------+|
X ->| |Imputer | ->|Scaler | ->| Model ||-> y_pred
    | +--------+  +--------+  +-------+|
    |                                  |
    +----------------------------------+
    
    pipe.fit(X_train, y_train)  # allena TUTTO
    pipe.predict(X_test)        # applica TUTTO
```

### Confronto: Senza vs Con Pipeline

| Senza Pipeline | Con Pipeline |
|----------------|--------------|
| Codice frammentato | Oggetto unico |
| Facile dimenticare step | Tutti gli step garantiti |
| Data leakage possibile | Data leakage impossibile |
| Difficile salvare/caricare | Un solo file da salvare |
| Cross-validation manuale | CV automatica e corretta |

### Sintassi Base

```python
from sklearn.pipeline import Pipeline

# Metodo 1: esplicito con nomi
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

# Metodo 2: senza nomi (make_pipeline)
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    LogisticRegression()
)
```

### Come Funziona Internamente

**Quando chiami pipe.fit(X, y):**
1. Ogni transformer chiama `fit_transform()` sui dati
2. L'output viene passato al passo successivo
3. L'ultimo step (modello) chiama solo `fit()`

**Quando chiami pipe.predict(X):**
1. Ogni transformer chiama solo `transform()` (NO fit!)
2. L'ultimo step chiama `predict()`

```
fit(X_train, y_train):
----------------------
X_train -> [Imputer.fit_transform] -> [Scaler.fit_transform] -> [Model.fit]

predict(X_test):
----------------
X_test -> [Imputer.transform] -> [Scaler.transform] -> [Model.predict] -> y_pred
```

---

## 1.3 ColumnTransformer: Pipeline per Dati Eterogenei

Nei dataset reali abbiamo spesso:
- **Colonne numeriche** -> SimpleImputer(mean) + StandardScaler
- **Colonne categoriche** -> SimpleImputer(most_frequent) + OneHotEncoder
- **Colonne ordinali** -> OrdinalEncoder
- **Colonne da ignorare** -> passthrough o drop

### Sintassi ColumnTransformer

```python
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ('name_1', transformer_1, columns_1),
    ('name_2', transformer_2, columns_2),
    ...
], remainder='drop')  # cosa fare con le altre colonne
```

### Parametro remainder

| Valore | Comportamento |
|--------|---------------|
| `'drop'` (default) | Scarta le colonne non specificate |
| `'passthrough'` | Mantiene le colonne non trasformate |
| transformer | Applica un transformer alle colonne rimanenti |

### Schema Completo con ColumnTransformer

```
+--------------------------------------------------------+
|                    ColumnTransformer                    |
|  +-----------------+    +------------------+            |
|  | numeric_pipe    |    | categorical_pipe |            |
|  |  Imputer(mean)  |    |  Imputer(freq)   |            |
|  |  StandardScaler |    |  OneHotEncoder   |            |
|  +--------+--------+    +--------+---------+            |
|           |                      |                      |
|           +----------+-----------+                      |
|                      | concatena                        |
|                      v                                  |
|            [features trasformate]                       |
+--------------------------------------------------------+
                       |
                       v
              +---------------+
              |    Modello    |
              | (RandomForest)|
              +---------------+
```

---

## 1.4 Pipeline con GridSearchCV

Quando usi GridSearchCV con una Pipeline, puoi ottimizzare parametri di **qualsiasi step**.

### Sintassi per Accedere ai Parametri

```python
# Con Pipeline esplicita
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

param_grid = {
    'svc__C': [0.1, 1, 10],        # stepname__param
    'svc__kernel': ['rbf', 'linear']
}

# Con make_pipeline (nomi automatici lowercase)
pipe = make_pipeline(StandardScaler(), SVC())

param_grid = {
    'standardscaler__with_mean': [True, False],
    'svc__C': [0.1, 1, 10]
}
```

---

## 1.5 Salvare e Caricare Pipeline

```python
import joblib

# Salva TUTTA la pipeline (preprocessing + modello)
joblib.dump(full_pipeline, 'model.pkl')

# Carica e predici su nuovi dati
loaded_pipe = joblib.load('model.pkl')
predictions = loaded_pipe.predict(new_data)
```

Un solo file contiene tutto il workflow!

# SEZIONE 2 - Mappa Mentale

## Flusso Decisionale: Quando Usare Pipeline

```
Ho piu di uno step di preprocessing?
            |
    +-------+-------+
    |               |
   NO              SI
    |               |
    v               v
Transformer      Farai training/prediction?
singolo OK              |
                +-------+-------+
                |               |
               NO              SI
                |               |
                v               v
          Step-by-step      USA PIPELINE!
          per EDA
```

## Checklist: Quando Pipeline e Obbligatoria

| Situazione | Pipeline? | Motivo |
|------------|-----------|--------|
| Cross-validation | SI | Previeni data leakage automaticamente |
| Hyperparameter tuning | SI | GridSearchCV lavora con pipeline |
| Modello in produzione | SI | Un solo oggetto da serializzare |
| Lavoro in team | SI | Codice piu leggibile e manutenibile |
| Solo EDA esplorativa | NO | Lavori step-by-step per capire |
| Debug preprocessing | NO | Isoli il problema senza pipeline |

## Tipi di Pipeline

```
1. PIPELINE SEMPLICE (solo numeriche)
+--------------------------------------------+
|  SimpleImputer -> StandardScaler -> Model  |
+--------------------------------------------+

2. PIPELINE CON COLUMNTRANSFORMER (dati misti)
+------------------------------------------------+
| ColumnTransformer                              |
|   +------------------+  +------------------+   |
|   | num_pipeline     |  | cat_pipeline     |   |
|   | Imputer->Scaler  |  | Imputer->OneHot  |   |
|   +------------------+  +------------------+   |
|               |                   |            |
|               +--------+----------+            |
|                        v                       |
+------------------------|------------------------+
                         v
                     [Model]
```

## Pattern da Ricordare

```
PATTERN 1: Pipeline Base
--------------------------
pipe = make_pipeline(
    SimpleImputer(),
    StandardScaler(),
    LogisticRegression()
)

PATTERN 2: Dati Misti
--------------------------
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

full_pipe = Pipeline([
    ('prep', preprocessor),
    ('model', RandomForest())
])

PATTERN 3: Con GridSearchCV
--------------------------
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_
```

# SEZIONE 3 - Quaderno Dimostrativo

In questa sezione vedremo 5 esercizi pratici:

1. **Esercizio 1**: Pipeline Base con Dati Numerici
2. **Esercizio 2**: ColumnTransformer per Dati Misti
3. **Esercizio 3**: Pipeline con GridSearchCV
4. **Esercizio 4**: Confronto Con/Senza Pipeline
5. **Esercizio 5**: Pipeline Completa per Produzione

In [1]:
# === ESERCIZIO 1: PIPELINE BASE CON DATI NUMERICI ===
# Perche: mostriamo come costruire una pipeline semplice per preprocessing + modello

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import make_classification

# Creiamo un dataset con valori mancanti
np.random.seed(42)
X, y = make_classification(
    n_samples=500,
    n_features=8,
    n_informative=5,
    n_redundant=2,
    flip_y=0.1,
    random_state=42
)

# Aggiungiamo 10% di valori mancanti
mask = np.random.random(X.shape) < 0.10
X[mask] = np.nan

print("="*60)
print("ESERCIZIO 1: Pipeline Base con Dati Numerici")
print("="*60)
print(f"\nDataset: {X.shape}")
print(f"Valori mancanti: {np.isnan(X).sum()} ({np.isnan(X).mean()*100:.1f}%)")

# Split dei dati
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Creiamo la pipeline: Imputer -> Scaler -> SVC
pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    SVC(kernel='rbf', random_state=42)
)

print(f"\nPipeline creata:")
print(pipe)

# Cross-validation (internamente applica fit solo su train di ogni fold)
cv_scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')

print(f"\nCross-Validation Results:")
print(f"  Scores per fold: {[f'{s:.3f}' for s in cv_scores]}")
print(f"  Mean Accuracy: {cv_scores.mean():.4f} +/- {cv_scores.std():.4f}")

# Fit finale e test
pipe.fit(X_train, y_train)
test_score = pipe.score(X_test, y_test)
print(f"\nTest Accuracy: {test_score:.4f}")

# --- MICRO-CHECKPOINT ---
assert cv_scores.mean() > 0.70, "CV accuracy troppo bassa"
assert test_score > 0.70, "Test accuracy troppo bassa"
print("\nMicro-checkpoint: Pipeline funziona correttamente!")

ESERCIZIO 1: Pipeline Base con Dati Numerici

Dataset: (500, 8)
Valori mancanti: 426 (10.7%)

Pipeline creata:
Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('svc', SVC(random_state=42))])

Cross-Validation Results:
  Scores per fold: ['0.870', '0.970', '0.880', '0.910', '0.860']
  Mean Accuracy: 0.8980 +/- 0.0397

Test Accuracy: 0.8960

Micro-checkpoint: Pipeline funziona correttamente!


In [2]:
# === ESERCIZIO 2: COLUMNTRANSFORMER PER DATI MISTI ===
# Perche: nei dataset reali abbiamo colonne numeriche e categoriche che richiedono preprocessing diverso

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

print("="*60)
print("ESERCIZIO 2: ColumnTransformer per Dati Misti")
print("="*60)

# Creiamo un dataset simil-Titanic con dati misti
np.random.seed(42)
n_samples = 600

df = pd.DataFrame({
    'Age': np.random.normal(30, 12, n_samples).clip(1, 80),
    'Income': np.abs(np.random.normal(50000, 20000, n_samples)),
    'CreditScore': np.random.randint(300, 850, n_samples).astype(float),
    'Employment': np.random.choice(['employed', 'self-employed', 'unemployed', 'retired'], n_samples),
    'Education': np.random.choice(['high_school', 'bachelor', 'master', 'phd'], n_samples),
    'HomeOwner': np.random.choice(['yes', 'no', 'rent'], n_samples)
})

# Target binario
df['Approved'] = ((df['Income'] > 40000) & (df['CreditScore'] > 600)).astype(int)

# Aggiungiamo valori mancanti realistici
df.loc[np.random.choice(n_samples, 50, replace=False), 'Age'] = np.nan
df.loc[np.random.choice(n_samples, 30, replace=False), 'Income'] = np.nan
df.loc[np.random.choice(n_samples, 20, replace=False), 'Employment'] = np.nan

print(f"\nDataset shape: {df.shape}")
print(f"\nValori mancanti:\n{df.isnull().sum()}")
print(f"\nDtypes:\n{df.dtypes}")

# Identifichiamo le colonne per tipo
numeric_features = ['Age', 'Income', 'CreditScore']
categorical_features = ['Employment', 'Education', 'HomeOwner']

# Pipeline per features numeriche
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline per features categoriche
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# ColumnTransformer che combina tutto
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Pipeline completa con modello
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

print("\nPipeline Completa:")
print(full_pipeline)

# Training e valutazione
X = df.drop('Approved', axis=1)
y = df['Approved']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

full_pipeline.fit(X_train, y_train)
accuracy = full_pipeline.score(X_test, y_test)

print(f"\nRisultati:")
print(f"  Train samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Test Accuracy: {accuracy:.4f}")

# Feature names dopo trasformazione
feature_names = preprocessor.get_feature_names_out()
print(f"\nFeatures trasformate ({len(feature_names)}):")
for i, name in enumerate(feature_names):
    print(f"  {i+1}. {name}")

# --- MICRO-CHECKPOINT ---
assert accuracy > 0.80, "Accuracy troppo bassa per dati misti"
assert len(feature_names) > len(numeric_features), "One-hot encoding non applicato"
print("\nMicro-checkpoint: ColumnTransformer funziona correttamente!")

ESERCIZIO 2: ColumnTransformer per Dati Misti

Dataset shape: (600, 7)

Valori mancanti:
Age            50
Income         30
CreditScore     0
Employment     20
Education       0
HomeOwner       0
Approved        0
dtype: int64

Dtypes:
Age            float64
Income         float64
CreditScore    float64
Employment      object
Education       object
HomeOwner       object
Approved         int64
dtype: object

Pipeline Completa:
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Income',
                                                   'CreditScore']

In [ ]:
# === ESERCIZIO 3: PIPELINE CON GRIDSEARCHCV ===
# Perche: possiamo ottimizzare iperparametri di qualsiasi step della pipeline

from sklearn.model_selection import GridSearchCV

print("="*60)
print("ESERCIZIO 3: Pipeline con GridSearchCV")
print("="*60)

# Dataset
np.random.seed(42)
X_grid, y_grid = make_classification(
    n_samples=400,
    n_features=10,
    n_informative=6,
    random_state=42
)

X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(
    X_grid, y_grid, test_size=0.25, random_state=42
)

print(f"Dataset: {X_grid.shape}")
print(f"Train: {X_train_g.shape}, Test: {X_test_g.shape}")

# Creiamo la pipeline
pipe_grid = make_pipeline(
    StandardScaler(),
    SVC(random_state=42)
)

print(f"\nPipeline:")
print(pipe_grid)

# Definiamo la griglia di iperparametri
# NOTA: con make_pipeline i nomi sono lowercase del transformer
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

print(f"\nGriglia iperparametri:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

# GridSearchCV
grid_search = GridSearchCV(
    pipe_grid,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

print(f"\nEseguendo GridSearchCV con 5-fold CV...")
grid_search.fit(X_train_g, y_train_g)

# Risultati
print(f"\nMiglior CV Score: {grid_search.best_score_:.4f}")
print(f"\nMigliori Parametri:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Test con migliori parametri
test_score_grid = grid_search.score(X_test_g, y_test_g)
print(f"\nTest Accuracy (best params): {test_score_grid:.4f}")

# Top 5 combinazioni
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values('rank_test_score')
print(f"\nTop 5 combinazioni:")
print(results[['rank_test_score', 'mean_test_score', 'std_test_score', 'params']].head())

# --- MICRO-CHECKPOINT ---
assert grid_search.best_score_ > 0.80, "Best CV score troppo basso"
assert test_score_grid > 0.80, "Test score troppo basso"
print("\nMicro-checkpoint: GridSearchCV ottimizza tutta la pipeline!")

In [ ]:
# === ESERCIZIO 4: CONFRONTO CON/SENZA PIPELINE ===
# Perche: dimostriamo che la pipeline produce risultati identici ma con codice piu pulito

print("="*60)
print("ESERCIZIO 4: Confronto Con/Senza Pipeline")
print("="*60)

# Dataset con valori mancanti
np.random.seed(42)
X_cmp, y_cmp = make_classification(
    n_samples=500,
    n_features=10,
    n_informative=6,
    flip_y=0.1,
    random_state=42
)

# Aggiungiamo valori mancanti
mask = np.random.random(X_cmp.shape) < 0.10
X_cmp[mask] = np.nan

# Split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cmp, y_cmp, test_size=0.25, random_state=42
)

print(f"Dataset: {X_cmp.shape}")
print(f"Valori mancanti: {np.isnan(X_cmp).sum()}")

# --- METODO 1: SENZA PIPELINE ---
print("\n" + "-"*40)
print("METODO 1: Senza Pipeline")
print("-"*40)

# Step 1: Imputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train_c)
X_train_imputed = imputer.transform(X_train_c)
X_test_imputed = imputer.transform(X_test_c)

# Step 2: Scaler
scaler = StandardScaler()
scaler.fit(X_train_imputed)
X_train_scaled = scaler.transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Step 3: Model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train_c)
score_no_pipe = model.score(X_test_scaled, y_test_c)

print(f"Test Accuracy: {score_no_pipe:.4f}")
print("Righe di codice: ~12")
print("Variabili intermedie: 6")
print("Oggetti da salvare: 3")

# --- METODO 2: CON PIPELINE ---
print("\n" + "-"*40)
print("METODO 2: Con Pipeline")
print("-"*40)

pipe_cmp = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipe_cmp.fit(X_train_c, y_train_c)
score_pipe = pipe_cmp.score(X_test_c, y_test_c)

print(f"Test Accuracy: {score_pipe:.4f}")
print("Righe di codice: 2")
print("Variabili intermedie: 0")
print("Oggetti da salvare: 1")

# Confronto
print("\n" + "-"*40)
print("CONFRONTO")
print("-"*40)
print(f"Risultati identici? {np.isclose(score_no_pipe, score_pipe)}")
print(f"Differenza: {abs(score_no_pipe - score_pipe):.6f}")

# --- MICRO-CHECKPOINT ---
assert np.isclose(score_no_pipe, score_pipe), "I risultati dovrebbero essere identici!"
print("\nMicro-checkpoint: Pipeline produce risultati identici con meno codice!")

In [ ]:
# === ESERCIZIO 5: PIPELINE COMPLETA PER PRODUZIONE ===
# Perche: mostriamo il workflow completo: preprocessing, tuning, salvataggio

import joblib
import os
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

print("="*60)
print("ESERCIZIO 5: Pipeline Completa per Produzione")
print("="*60)

# Creiamo dataset con dati misti
np.random.seed(42)
n = 800

df_prod = pd.DataFrame({
    'feature_1': np.random.normal(0, 1, n),
    'feature_2': np.random.exponential(2, n),
    'feature_3': np.random.randint(0, 100, n).astype(float),
    'category_1': np.random.choice(['A', 'B', 'C'], n),
    'category_2': np.random.choice(['X', 'Y'], n)
})

df_prod['target'] = (
    0.5 * df_prod['feature_1'] + 
    0.3 * df_prod['feature_2'] + 
    0.2 * (df_prod['category_1'] == 'A').astype(float)
) > 0.5
df_prod['target'] = df_prod['target'].astype(int)

# Valori mancanti
df_prod.loc[np.random.choice(n, 40, replace=False), 'feature_1'] = np.nan
df_prod.loc[np.random.choice(n, 25, replace=False), 'category_1'] = np.nan

print(f"Dataset: {df_prod.shape}")
print(f"Valori mancanti: {df_prod.isnull().sum().sum()}")

# Definiamo le colonne
num_cols = ['feature_1', 'feature_2', 'feature_3']
cat_cols = ['category_1', 'category_2']

# Pipeline per numeriche
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline per categoriche
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Preprocessor
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Pipeline completa
production_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Split
X_prod = df_prod.drop('target', axis=1)
y_prod = df_prod['target']

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_prod, y_prod, test_size=0.25, random_state=42
)

# Hyperparameter tuning
param_grid_prod = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, None]
}

grid_prod = GridSearchCV(
    production_pipeline,
    param_grid_prod,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

print("\nEseguendo GridSearchCV...")
grid_prod.fit(X_train_p, y_train_p)

print(f"\nMiglior CV Score: {grid_prod.best_score_:.4f}")
print(f"Migliori Parametri: {grid_prod.best_params_}")

# Best model
best_pipeline = grid_prod.best_estimator_
test_score_prod = best_pipeline.score(X_test_p, y_test_p)
print(f"Test Accuracy: {test_score_prod:.4f}")

# Salvare il modello (demo)
model_path = 'production_model.pkl'
joblib.dump(best_pipeline, model_path)
print(f"\nModello salvato: {model_path}")
print(f"Dimensione file: {os.path.getsize(model_path) / 1024:.1f} KB")

# Simulare caricamento e predizione su nuovi dati
loaded_pipeline = joblib.load(model_path)
new_data = pd.DataFrame({
    'feature_1': [0.5, -0.3],
    'feature_2': [2.1, 1.5],
    'feature_3': [50, 75],
    'category_1': ['A', 'B'],
    'category_2': ['X', 'Y']
})

predictions = loaded_pipeline.predict(new_data)
probabilities = loaded_pipeline.predict_proba(new_data)

print(f"\nPredizioni su nuovi dati:")
print(f"  Predizioni: {predictions}")
print(f"  Probabilita classe 1: {probabilities[:, 1]}")

# Pulizia
os.remove(model_path)
print(f"\nFile {model_path} rimosso (era solo una demo)")

# --- MICRO-CHECKPOINT ---
assert test_score_prod > 0.70, "Test score troppo basso"
assert len(predictions) == 2, "Dovrebbero esserci 2 predizioni"
print("\nMicro-checkpoint: Pipeline pronta per produzione!")

# SEZIONE 4 - Metodi Principali

## Pipeline

| Metodo | Sintassi | Descrizione |
|--------|----------|-------------|
| `Pipeline()` | `Pipeline([('name', transformer), ...])` | Crea pipeline con step nominati |
| `make_pipeline()` | `make_pipeline(t1, t2, t3)` | Shortcut: genera nomi automatici |
| `pipe.fit()` | `pipe.fit(X, y)` | Esegue fit_transform su tutti gli step |
| `pipe.predict()` | `pipe.predict(X)` | Esegue transform + predict |
| `pipe.score()` | `pipe.score(X, y)` | Esegue transform + score |
| `pipe['name']` | `pipe['scaler']` | Accede a uno step per nome |
| `pipe.named_steps` | `pipe.named_steps['scaler']` | Dizionario degli step |
| `pipe.set_params()` | `pipe.set_params(scaler__with_mean=False)` | Modifica parametri |

## ColumnTransformer

| Metodo | Sintassi | Descrizione |
|--------|----------|-------------|
| `ColumnTransformer()` | `ColumnTransformer([('name', tr, cols)])` | Trasformazioni per colonne diverse |
| `remainder` | `remainder='drop'\|'passthrough'` | Gestione colonne non specificate |
| `ct.fit_transform()` | `ct.fit_transform(X)` | Fit e trasforma le colonne |
| `ct.get_feature_names_out()` | `ct.get_feature_names_out()` | Nomi feature dopo trasformazione |
| `make_column_selector()` | `make_column_selector(dtype_include=np.number)` | Seleziona colonne per dtype |

## GridSearchCV con Pipeline

| Sintassi | Descrizione |
|----------|-------------|
| `'stepname__param'` | Accesso parametri: nome step + doppio underscore |
| `grid.best_estimator_` | La pipeline con migliori parametri |
| `grid.best_params_` | Dizionario dei migliori parametri |
| `grid.cv_results_` | Risultati dettagliati di tutti i trial |

## Persistenza

| Metodo | Sintassi | Descrizione |
|--------|----------|-------------|
| `joblib.dump()` | `joblib.dump(pipe, 'model.pkl')` | Salva pipeline su disco |
| `joblib.load()` | `joblib.load('model.pkl')` | Carica pipeline da disco |

# SEZIONE 5 - Glossario

| Termine | Definizione |
|---------|-------------|
| **Pipeline** | Sequenza ordinata di trasformazioni + modello in un unico oggetto |
| **ColumnTransformer** | Applica trasformazioni diverse a gruppi di colonne |
| **Data Leakage** | Errore: usare informazioni dal test set durante il training |
| **make_pipeline** | Shortcut per creare Pipeline con nomi automatici |
| **fit_transform** | Combina fit e transform in una sola chiamata |
| **transform** | Applica trasformazione senza ricalcolare i parametri |
| **Transformer** | Oggetto che implementa fit e transform (es. StandardScaler) |
| **Estimator** | Oggetto che implementa fit e predict (es. modello) |
| **Step** | Un singolo componente della pipeline |
| **remainder** | Parametro per gestire colonne non specificate in ColumnTransformer |
| **passthrough** | Mantiene le colonne senza trasformazioni |
| **handle_unknown** | Parametro OneHotEncoder per gestire categorie nuove |
| **sparse_output** | Parametro per controllare se output e sparso o denso |
| **named_steps** | Attributo per accedere agli step per nome |
| **best_estimator_** | La pipeline fittata con i migliori parametri da GridSearchCV |
| **stepname__param** | Sintassi per accedere ai parametri degli step |
| **joblib** | Libreria per serializzare oggetti Python (pipeline) |
| **make_column_selector** | Helper per selezionare colonne per dtype |
| **n_jobs** | Parametro per parallelizzazione (-1 = tutti i core) |
| **cv_results_** | Dizionario con risultati dettagliati di GridSearchCV |

# SEZIONE 6 - Errori Comuni

| N. | Errore | Problema | Soluzione |
|----|--------|----------|-----------|
| 1 | Preprocessing prima dello split | Data leakage: scaler vede anche test | Usare Pipeline con CV |
| 2 | fit_transform su test | Il test non deve influenzare i parametri | Solo transform su test (Pipeline lo fa automaticamente) |
| 3 | Dimenticare handle_unknown | Crash con categorie nuove in produzione | `OneHotEncoder(handle_unknown='ignore')` |
| 4 | Nome step sbagliato in GridSearchCV | Parametri non trovati | Controllare nomi con `pipe.named_steps.keys()` |
| 5 | Doppio underscore dimenticato | `stepname_param` invece di `stepname__param` | Sempre doppio underscore per accedere ai parametri |
| 6 | remainder='drop' non voluto | Colonne perse silenziosamente | Specificare `remainder='passthrough'` se servono |
| 7 | Salvare modello e preprocessor separati | Rischio disallineamento | Salvare tutta la Pipeline con joblib |
| 8 | Ordine step sbagliato | Scaler prima di Imputer fallisce con NaN | Prima Imputer, poi Scaler |
| 9 | sparse_output=True con modelli incompatibili | Errori con alcuni classificatori | `sparse_output=False` per sicurezza |
| 10 | Cross-validation senza Pipeline | Data leakage nel preprocessing | Sempre Pipeline quando fai CV |

# SEZIONE 7 - Conclusione

## Concetti Chiave Appresi

1. **Pipeline** incapsula preprocessing + modello in un unico oggetto
2. **Data Leakage** si previene automaticamente usando Pipeline con cross-validation
3. **ColumnTransformer** permette trasformazioni diverse per colonne numeriche e categoriche
4. **make_pipeline** e uno shortcut che genera nomi automatici
5. **GridSearchCV** ottimizza iperparametri di qualsiasi step della pipeline
6. **joblib** permette di salvare/caricare l'intera pipeline per produzione

## Formula Mentale

```
PIPELINE = Sequenza(Transformer1, Transformer2, ..., Estimator)

fit(X_train) = foreach step: fit_transform() fino all'ultimo che fa fit()
predict(X_new) = foreach step: transform() fino all'ultimo che fa predict()
```

## Prossima Lezione

**Lezione 14**: Alberi Decisionali e Random Forest
- Come funziona un Decision Tree
- Gini vs Entropy
- Ensemble: Bagging e Random Forest
- Feature Importance

# SEZIONE 8 - Checklist

## Prima di usare Pipeline

- [ ] Ho identificato tutti gli step di preprocessing necessari
- [ ] Ho verificato l'ordine corretto degli step (Imputer prima di Scaler)
- [ ] Ho separato colonne numeriche da categoriche (se presenti)

## Durante la costruzione

- [ ] Ho usato make_pipeline o Pipeline con nomi descrittivi
- [ ] Ho impostato `handle_unknown='ignore'` per OneHotEncoder
- [ ] Ho impostato `sparse_output=False` se necessario
- [ ] Ho specificato `remainder` in ColumnTransformer

## Per GridSearchCV

- [ ] Ho usato la sintassi `stepname__param` per i parametri
- [ ] Ho verificato i nomi degli step con `pipe.named_steps.keys()`
- [ ] Ho impostato `n_jobs=-1` per parallelizzazione

## Per produzione

- [ ] Ho salvato l'intera pipeline con joblib (non solo il modello)
- [ ] Ho testato load + predict su dati nuovi
- [ ] Ho verificato che il file salvato sia ragionevole in dimensione

# SEZIONE 9 - Changelog

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-15 | Creazione lezione su Pipeline di Scikit-Learn |
| 1.1 | 2024-01-15 | Ristrutturazione con template 9 sezioni |
| 1.2 | 2024-01-15 | Aggiunta 5 esercizi dimostrativi con micro-checkpoint |
| 1.3 | 2024-01-15 | Aggiunta sezione ColumnTransformer e GridSearchCV |